In [2]:
class Processus:
    """Classe définissant un processus"""
    def __init__(self, pid, duree):
        self.pid = pid
        self._duree = duree  # ne pas modifier cet attribut

    def execute(self, quantum):
        """Diminue la durée d'exécution restante de quantum. Bloque à 0"""
        self._duree = max(0, self._duree - quantum)

    def est_fini(self):
        """Renvoie le booléen répondant à la question 'Le processus est-il terminé ?'"""
        return self._duree == 0

    def __repr__(self):
        """Affichage"""
        return f"(PID={self.pid} ; durée={self._duree})"

In [1]:
from collections import deque

class File:
    """Classe définissant une structure de file"""

    def __init__(self):
        self.valeurs = deque([])

    def est_vide(self):
        """Renvoie le booléen True si la file est vide, False sinon"""
        return len(self.valeurs) == 0

    def enfile(self, x):
        """Place x à la queue de la file"""
        self.valeurs.appendleft(x)

    def defile(self):
        """Retire et renvoie l'élément placé à la tête de la file.
        Provoque une erreur si la file est vide
        """
        if self.est_vide():
            raise ValueError("La file est vide")
        return self.valeurs.pop()

    def __repr__(self):
        """Convertit la file en une chaîne de caractères"""
        return f"{list(self.valeurs)}"

Processus et tourniquet:

In [14]:
def tourniquet(file, quantum):
    ordre = []
    while not file.est_vide():
        processus = file.defile()
        ordre.append(processus.pid)
        processus.execute(quantum)
        if not processus.est_fini():
            file.enfile(processus)
    return ordre

In [15]:
f = File()

In [16]:
f.enfile(Processus(0, 5))

In [17]:
f.enfile(Processus(1, 2))

In [18]:
print(tourniquet(f, 1))

[0, 1, 0, 1, 0, 0, 0]


Etat d'un processus